# PyTorch - homework 1


Please run the whole notebook with your code and submit the `.ipynb` file that includes your answers. 

In [1]:
from termcolor import colored

student_number="1002949"
student_name="Hong Pengfei"

print(colored("Homework by "  + student_name + ', number: ' + student_number,'red'))

Homework by Hong Pengfei, number: 1002949


 ## Question 1 -- matrix multiplication

Implement the following mathematical operation on both the CPU and GPU (use Google Colab or another cloud service if you don't have a GPU in your computer). Print:

a) which type of GPU card you have and 

b) show the computation time for both CPU and GPU (using PyTorch). 

c) How much % fast is the GPU? 

 The operation to implement is the dot product $C = B * A^T$

 whereby $A$ is a random matrix of size $30,000 \times 1000$ and $B$ is a random matrix of size $3000 \times 1000$. In addition to the required information asked above:
 
 d) please also print the resulting two $C$ matrices (they should be the same btw). 
 



In [2]:
# implement solution here
import torch
print(colored(f"a) GPU card type: f{torch.cuda.get_device_name(0)}", 'blue'))

a) GPU card type: fGeForce GTX TITAN X


In [3]:
A = torch.randn(30_000, 1_000, dtype=torch.float64); B = torch.randn(3_000, 1_000, dtype=torch.float64)
print("CPU")
%timeit C = B @ A.T
C_cpu = B@A.T
print("GPU")
A = A.cuda(); B = B.cuda()
%timeit C = B @ A.T
C_cuda = B@A.T

CPU
365 ms ± 105 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
GPU
37.9 µs ± 25.1 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


b)

CPU running time: 365ms

GPU running time: 37.9 $\mu$s

In [4]:
import numpy as np; print(f"c) GPU is {np.round(365e-3/37.9e-6)} faster than CPU")

c) GPU is 9631.0 faster than CPU


In [14]:
print("CPU")
print(C_cpu)
print('='*80)
print("GPU")
print(C_cuda)
assert (C_cpu.numpy().astype(np.float16) == C_cuda.cpu().numpy().astype(np.float16)).all()
print("d) they are equal")

CPU
tensor([[ 26.5482,  32.8680,  61.0703,  ...,  15.0638,  -5.6526, -16.7337],
        [ 15.7892,   7.0094, -39.8665,  ...,  20.9929, -19.3506,  58.0912],
        [ 20.1616, -34.0288,  -9.1990,  ..., -17.1027,  19.2191, -13.3171],
        ...,
        [-20.8380, -47.9716, -38.1586,  ...,  18.0809, -20.0048,  33.7353],
        [  5.8443, -41.5327,   0.9640,  ...,  -0.4082,   9.2285, -15.4392],
        [-38.9589,  59.0055,  19.9296,  ..., -17.2140,  -1.6591,  -6.5475]],
       dtype=torch.float64)
GPU
tensor([[ 26.5482,  32.8680,  61.0703,  ...,  15.0638,  -5.6526, -16.7337],
        [ 15.7892,   7.0094, -39.8665,  ...,  20.9929, -19.3506,  58.0912],
        [ 20.1616, -34.0288,  -9.1990,  ..., -17.1027,  19.2191, -13.3171],
        ...,
        [-20.8380, -47.9716, -38.1586,  ...,  18.0809, -20.0048,  33.7353],
        [  5.8443, -41.5327,   0.9640,  ...,  -0.4082,   9.2285, -15.4392],
        [-38.9589,  59.0055,  19.9296,  ..., -17.2140,  -1.6591,  -6.5475]],
       device='cuda:0', 

## Question 2 - grad


Find the gradient (partial derivatives) of the function $g(w)$ below. 

Let  $w=[w_1,w_2]^T$

Consider  $g(w)=2w_1w_2+w_2cos(w_1)$

a) In PyTorch, compute:   $\Delta_w g(w)$ 

 and verify that $\Delta_w g([\pi,1])=[2,2\pi−1]^T$ using the grad function, whereby the first position is the partial for $w_1$ and the second position is the partial for $w_2$. 

b) You can also write a function to manually calculate these partial derivatives! You can review your differential equations math at [here](https://www.wolframalpha.com/input/?i=derivative+y+cos%28x%29) and implement this is a second function below to verify that it comes to the same solution. 


In [29]:
w = torch.tensor([np.pi, 1], requires_grad=True)
def g(w): return 2*w[0]*w[1] + w[1]*torch.cos(w[0])
g_w = g(w)
g_w.backward()
print(f"a) gradient using pytorch: {np.round(w.grad.data.numpy(), 3)}")
print(f"\t  standard answer: {[2, np.round(2*np.pi-1, 3)]}")

a) gradient using pytorch: [2.    5.283]
	  standard answer: [2, 5.283]


In [30]:
def partial_g(w): return [-(torch.sin(w[0])-2)*w[1], 2*w[0]+torch.cos(w[0])]
w = torch.tensor([np.pi, 1])
print(f"gradient using Wolfram: {partial_g(w)}")

gradient using Wolfram: [tensor(2.), tensor(5.2832)]


## Question 3 - dance hit song prediction

Implement logistic regression in PyTorch for the following dance hit song prediction training dataset: 
https://dorax.s3.ap-south-1.amazonaws.com/herremans_hit_1030training.csv

 * Input variables: a number of audio features (most already standardized so don't worry about that)
 * Target variable: Topclass1030: 
   * 1 means it was a top 10 hit song; 
   * 0 means it never went above top 30 position.

This dataset is derived from my paper on dance hit song prediction, for full description of features have a look at https://arxiv.org/abs/1905.08076. 

Print the evolution of the loss every few epochs and train the model until it converges. 
 
 After training the logistic regression model, calculate the prediction accuracy on the test set: 
 https://dorax.s3.ap-south-1.amazonaws.com/herremans_hit_1030test.csv








In [27]:
# Your code here
import torch
import pandas as pd, numpy as np
from sklearn.model_selection import train_test_split
from fastprogress import progress_bar
# load data
ds_trn = pd.read_csv("https://dorax.s3.ap-south-1.amazonaws.com/herremans_hit_1030training.csv")
ds_trn, ds_val = train_test_split(ds_trn.values, test_size=0.3)
x_trn, y_trn = torch.Tensor(ds_trn[:,:-1]).cuda(), torch.Tensor(ds_trn[:,-1]).cuda()
x_val, y_val = torch.Tensor(ds_val[:,:-1]).cuda(), torch.Tensor(ds_val[:,-1]).cuda()

# define logistic regression model
logreg_clf = torch.nn.Sequential(torch.nn.Linear(len(x_trn[0]), 1), torch.nn.Sigmoid()).cuda()
loss_fn = torch.nn.BCELoss(); optimizer = torch.optim.Adam(logreg_clf.parameters(), lr=1e-4)
# train model
epoch = 1000
best_loss = np.inf
patient = 0
for i in progress_bar(range(epoch)):
    y_pred = logreg_clf(x_trn)
    loss = loss_fn(y_pred.squeeze(), y_trn)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if i % 100 == 0:
        val_loss = torch.nn.BCELoss()(logreg_clf(x_val), y_val.squeeze())
        print(f"epoch: {i} \t train_loss: {loss.data} \t val_loss: {val_loss}")
        if val_loss < best_loss:
            best_loss = val_loss
            from copy import deepcopy
            best_params = deepcopy(logreg_clf.state_dict())
        else:
            patient += 1
        if patient > 2: print("Early Stopping"); break
logreg_clf.load_state_dict(best_params)

epoch: 0 	 train_loss: 0.70167475938797 	 val_loss: 0.68543541431427
epoch: 100 	 train_loss: 0.6786022186279297 	 val_loss: 0.6548535227775574
epoch: 200 	 train_loss: 0.6609829068183899 	 val_loss: 0.6341501474380493
epoch: 300 	 train_loss: 0.6470436453819275 	 val_loss: 0.6193217635154724
epoch: 400 	 train_loss: 0.6357281804084778 	 val_loss: 0.608698308467865
epoch: 500 	 train_loss: 0.6262457966804504 	 val_loss: 0.6012091636657715
epoch: 600 	 train_loss: 0.618064284324646 	 val_loss: 0.5959230065345764
epoch: 700 	 train_loss: 0.6108546853065491 	 val_loss: 0.5920870900154114
epoch: 800 	 train_loss: 0.6044106483459473 	 val_loss: 0.5891606211662292
epoch: 900 	 train_loss: 0.598591685295105 	 val_loss: 0.5867881178855896


<All keys matched successfully>

Run the below code to test the accuracy of your model on the training set: 

In [28]:
import pandas as pd 

test = pd.read_csv('https://dorax.s3.ap-south-1.amazonaws.com/herremans_hit_1030test.csv')
labels = test.iloc[:,-1]
test = test.drop('Topclass1030', axis=1)
testdata = torch.Tensor(test.values).cuda()
testlabels = torch.Tensor(labels.values).view(-1,1).cuda()

TP = 0
TN = 0
FN = 0
FP = 0

for i in range(0, testdata.size()[0]): 
  # print(testdata[i].size())
    Xtest = testdata[i]
    y_hat = logreg_clf(Xtest)
  
    if y_hat > 0.5:
        prediction = 1
    else: 
        prediction = 0

    if (prediction == testlabels[i]):
        if (prediction == 1):
            TP += 1
        else: 
            TN += 1

    else:
        if (prediction == 1):
            FP += 1
        else: 
            FN += 1

print("True Positives: {0}, True Negatives: {1}".format(TP, TN))
print("False Positives: {0}, False Negatives: {1}".format(FP, FN))
rate = TP/(FN+TP)
print("Class specific accuracy of correctly predicting a hit song is {0}".format(rate))

True Positives: 47, True Negatives: 14
False Positives: 15, False Negatives: 3
Class specific accuracy of correctly predicting a hit song is 0.94
